# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
#model = timm.create_model('efficientnet_b1', pretrained=True)
#3, 320, 320
#name: (channel_multiplier, depth_multiplier, resolution, dropout_rate)
#(1.4, 1.8, 380, 0.4)

In [5]:
#model.blocks[0][0]

## load config and set logger

In [6]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [7]:
# Setting seed
modeler.set_seed(42)

In [8]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/DCASE_experiments/SSL-Efficientnet_Barlow_Twins/dcase2021_task2/config.yaml'

In [9]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [10]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths + dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths + add_dev_target_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3026, source_size : 3105, target_size : 9
Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [12]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [11]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('FEATURE EXTRACTION')
    net = Model(n_out=config['param']['mel_bins'])
    optimizer = optim.Adam(net.parameters(), lr=config['param']['lr'])
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, steps_per_epoch=len(dataloaders_dict['train']), epochs=config['param']['num_epochs'])
    output_dicts = modeler.train_net(net, dataloaders_dict, writer, optimizer, scheduler)
    
    out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    torch.save(output_dicts['net'], out_path)
    logger.info(f'SAVE SUCCESS : {out_path}')
    
    com.toc()

# run

In [ ]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-08-12 16:57:45,823 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-08-12 16:57:45,824 - 00_train.py - INFO - MAKE DATA_LOADER
2021-08-12 16:57:45,825 - 00_train.py - INFO - FEATURE EXTRACTION


use: cuda:0


100%|██████████| 48/48 [00:31<00:00,  1.55it/s]
2021-08-12 16:58:19,984 - pytorch_modeler.py - INFO - epoch:1/300, tr_loss:64193.139282
100%|██████████| 48/48 [00:30<00:00,  1.59it/s]
2021-08-12 16:58:50,160 - pytorch_modeler.py - INFO - epoch:2/300, tr_loss:62613.353638
100%|██████████| 48/48 [00:30<00:00,  1.60it/s]
2021-08-12 16:59:20,220 - pytorch_modeler.py - INFO - epoch:3/300, tr_loss:59394.322876
100%|██████████| 48/48 [00:29<00:00,  1.61it/s]
2021-08-12 16:59:49,974 - pytorch_modeler.py - INFO - epoch:4/300, tr_loss:56557.232422
100%|██████████| 48/48 [00:30<00:00,  1.58it/s]
2021-08-12 17:00:20,355 - pytorch_modeler.py - INFO - epoch:5/300, tr_loss:55033.081909
100%|██████████| 48/48 [00:30<00:00,  1.59it/s]
2021-08-12 17:00:50,495 - pytorch_modeler.py - INFO - epoch:6/300, tr_loss:52975.909424
100%|██████████| 48/48 [00:30<00:00,  1.59it/s]
2021-08-12 17:01:20,777 - pytorch_modeler.py - INFO - epoch:7/300, tr_loss:50493.723755
100%|██████████| 48/48 [00:29<00:00,  1.60it/s]


100%|██████████| 48/48 [00:28<00:00,  1.68it/s]
2021-08-12 17:27:40,727 - pytorch_modeler.py - INFO - epoch:61/300, tr_loss:38090.017761
100%|██████████| 48/48 [00:28<00:00,  1.66it/s]
2021-08-12 17:28:09,616 - pytorch_modeler.py - INFO - epoch:62/300, tr_loss:36600.686890
100%|██████████| 48/48 [00:28<00:00,  1.68it/s]
2021-08-12 17:28:38,233 - pytorch_modeler.py - INFO - epoch:63/300, tr_loss:35801.614197
100%|██████████| 48/48 [00:28<00:00,  1.68it/s]
2021-08-12 17:29:06,874 - pytorch_modeler.py - INFO - epoch:64/300, tr_loss:35779.560181
100%|██████████| 48/48 [00:30<00:00,  1.59it/s]
2021-08-12 17:29:36,994 - pytorch_modeler.py - INFO - epoch:65/300, tr_loss:35850.868958
100%|██████████| 48/48 [00:31<00:00,  1.50it/s]
2021-08-12 17:30:08,909 - pytorch_modeler.py - INFO - epoch:66/300, tr_loss:35149.927307
100%|██████████| 48/48 [00:31<00:00,  1.51it/s]
2021-08-12 17:30:40,714 - pytorch_modeler.py - INFO - epoch:67/300, tr_loss:36438.966125
100%|██████████| 48/48 [00:30<00:00,  1.5